In [11]:
import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
from scipy.stats import pearsonr, spearmanr

from shapely.geometry import Point
from shapely.ops import unary_union

Update the equity index values from 2024 to 2025

In [12]:
df_equity_2025 = pd.read_excel('../ct-data/equity-census-2025.xlsx')
gdf_equity_map_2024 = gpd.read_file('../ct-data/equity-2024-layer.geo.json')

In [13]:
# Force both keys to the same string format with 2 decimal places
df_equity_2025["Census tract"] = df_equity_2025["Census tract"].astype(float).map("{:.2f}".format)
gdf_equity_map_2024["ctuid"] = gdf_equity_map_2024["ctuid"].astype(float).map("{:.2f}".format)

# Rename columns for consistency
df_equity_2025.rename(columns={"Census tract": "ctuid"}, inplace=True)
gdf_equity_map_2024 = gdf_equity_map_2024.rename(columns={"Equity Index": "Equity Index 2024"})

# Select and rename columns to merge
columns_to_merge = {
    "Normalized Equity Score": "Equity Index",
    "Eviction rate": "%Evic",
    "Unemployment rate": "%Unemp",
    "  % No certificate, diploma or degree": "%NoEdu",
    "Gov transfer": "%IncomeGT"
}

new_census_vars = list(columns_to_merge.keys())[1:]
df_equity_2025[new_census_vars] = df_equity_2025[new_census_vars] * 100
df_equity_2025[new_census_vars] = df_equity_2025[new_census_vars].round(2)

df_to_merge = df_equity_2025[list(columns_to_merge.keys()) + ["ctuid"]]
df_to_merge = df_to_merge.rename(columns=columns_to_merge)

# Merge data frames on ctuid
gdf_equity_map_2025 = gdf_equity_map_2024.merge(df_to_merge, on="ctuid", how="left")

# Reorder columns so 'Equity Index' comes before 'Equity Index 2024' and 'geometry' is the last column
columns_order = [
    'ctuid', 'Equity Index', 'Equity Index 2024'
] + [col for col in gdf_equity_map_2025.columns if col not in ['ctuid', 'Equity Index', 'Equity Index 2024', 'geometry']] + ['geometry']
gdf_equity_map_2025 = gdf_equity_map_2025[columns_order]

gdf_equity_map_2025.to_file("../ct-data/equity-2025-layer.geo.json", driver="GeoJSON")

Identify the breakpoints for quintiles and investigate composition

In [14]:
gdf_ct = gpd.read_file('../ct-data/equity-2025-layer.geo.json')

# Calculate quintile breakpoints
quintiles = np.quantile(gdf_ct['Equity Index'].dropna(), [0, 0.2, 0.4, 0.6, 0.8, 1])
print("Quintile breakpoints:", quintiles)

Quintile breakpoints: [0.         0.2985527  0.37396513 0.43815575 0.52905651 1.        ]


In [15]:
gdf_ct = gpd.read_file('../ct-data/equity-2025-layer.geo.json')
gdf_ct = gdf_ct[['ctuid', 'Equity Index 2024', 'Equity Index']].dropna()

# Compute Pearson and Spearman correlations
pearson_corr, _ = pearsonr(gdf_ct['Equity Index 2024'], gdf_ct['Equity Index'])
spearman_corr, _ = spearmanr(gdf_ct['Equity Index 2024'], gdf_ct['Equity Index'])

print(f"Pearson correlation: {pearson_corr}")
print(f"Spearman correlation: {spearman_corr}")

Pearson correlation: 0.8035789674161269
Spearman correlation: 0.7887300652384908


In [16]:
# Create quintile transition matrix
gdf_ct['Quintile_old'] = pd.qcut(gdf_ct['Equity Index 2024'], q=5, labels=["Q1_old", "Q2_old", "Q3_old", "Q4_old", "Q5_old"])
gdf_ct['Quintile_new'] = pd.qcut(gdf_ct['Equity Index'], q=5, labels=["Q1_new", "Q2_new", "Q3_new", "Q4_new", "Q5_new"])

# Initialize transition matrix
quintile_labels = ["Q1_old", "Q2_old", "Q3_old", "Q4_old", "Q5_old"]
transition_matrix = pd.DataFrame(0.0, index=quintile_labels, columns=["Q1_new", "Q2_new", "Q3_new", "Q4_new", "Q5_new"])

# Calculate percentages for transitions
for old_quintile in quintile_labels:
    subset = gdf_ct[gdf_ct['Quintile_old'] == old_quintile]
    total_count = len(subset)
    if total_count > 0:
        for new_quintile in ["Q1_new", "Q2_new", "Q3_new", "Q4_new", "Q5_new"]:
            count_in_new = (subset['Quintile_new'] == new_quintile).sum()
            transition_matrix.at[old_quintile, new_quintile] = (count_in_new / total_count) * 100

# Save the transition matrix to a CSV file
transition_matrix.to_csv('../ct-data/quintile-composition.csv', index=True)

print(transition_matrix)

           Q1_new     Q2_new     Q3_new     Q4_new     Q5_new
Q1_old  63.849765  25.352113   9.859155   0.938967   0.000000
Q2_old  25.471698  35.377358  25.000000  13.679245   0.471698
Q3_old   8.333333  22.222222  34.259259  30.092593   5.092593
Q4_old   1.923077  12.500000  21.153846  37.500000  26.923077
Q5_old   0.471698   4.245283   9.433962  17.924528  67.924528


Count the number of locations for each census tract as well as an 800m buffer. Further, tag each census tract by the census division

In [17]:
# Read data
gdf_locations = gpd.read_file('../joined-data/simplified_matches_4326_full.geo.json')
gdf_ct = gpd.read_file('../ct-data/equity-2025-layer.geo.json')
gdf_gta = gpd.read_file('../ct-data/GTA_CD.gpkg').to_crs(3857)
gdf_gta = gdf_gta[gdf_gta['CDNAME'].isin(['Toronto', 'Peel', 'York'])]

# Change CRS to 3857 for accurate distance calculations
gdf_locations = gdf_locations.to_crs(epsg=3857)
gdf_ct = gdf_ct.to_crs(epsg=3857)
gdf_gta = gdf_gta.to_crs(epsg=3857)

In [18]:
def get_region_with_max_overlap(tract, gta):
    overlaps = gta[gta.intersects(tract.geometry)]
    if not overlaps.empty:
        overlaps['overlap_area'] = overlaps.geometry.intersection(tract.geometry).area
        return overlaps.loc[overlaps['overlap_area'].idxmax(), 'CDNAME']
    return None

gdf_ct['region'] = gdf_ct.apply(lambda row: get_region_with_max_overlap(row, gdf_gta), axis=1)

In [19]:
# Initialize new columns
for col in ['own_count', 'rent_count', 'unknown_count', 'total_count',
            'own_count_800m', 'rent_count_800m', 'unknown_count_800m', 'total_count_800m']:
    gdf_ct[col] = 0

# Iterate over census tracts
for idx, tract in tqdm(gdf_ct.iterrows(), total=len(gdf_ct)):
    # Locations within the census tract
    locations_in_tract = gdf_locations[gdf_locations.within(tract.geometry)]
    
    # Count tenure categories
    gdf_ct.at[idx, 'own_count'] = (locations_in_tract['Tenure'] == 'Own').sum()
    gdf_ct.at[idx, 'rent_count'] = (locations_in_tract['Tenure'] == 'Rent').sum()
    gdf_ct.at[idx, 'unknown_count'] = (locations_in_tract['Tenure'] == 'Unknown').sum()
    gdf_ct.at[idx, 'total_count'] = len(locations_in_tract)

    # Buffer geometry for 800m radius around the entire tract
    buffer_800m = tract.geometry.buffer(800)
    locations_in_buffer = gdf_locations[gdf_locations.geometry.intersects(buffer_800m)]

    # centroid = tract.geometry.centroid
    # buffer_800m = centroid.buffer(800)
    # locations_in_buffer = gdf_locations[gdf_locations.geometry.intersects(buffer_800m)]
    
    # Count tenure categories within 800m
    gdf_ct.at[idx, 'own_count_800m'] = (locations_in_buffer['Tenure'] == 'Own').sum()
    gdf_ct.at[idx, 'rent_count_800m'] = (locations_in_buffer['Tenure'] == 'Rent').sum()
    gdf_ct.at[idx, 'unknown_count_800m'] = (locations_in_buffer['Tenure'] == 'Unknown').sum()
    gdf_ct.at[idx, 'total_count_800m'] = len(locations_in_buffer)

# Drop the original columns
gdf_ct = gdf_ct.drop(columns=['Own_count', 'Rent_count', 'Unknown_count', 'Total_count'])

100%|██████████| 1065/1065 [00:03<00:00, 337.13it/s]


In [20]:
# Revert CRS back to 4326 before saving
gdf_ct = gdf_ct.to_crs(epsg=4326)

# Save updated GeoDataFrame
gdf_ct.to_file('../ct-data/ct-data-all.geo.json', driver='GeoJSON')